#`About Dataset`#
#input:Incorrect Sentence
#target:Correct Sentence

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

In [6]:
Grammaratical_Error=pd.read_csv("/kaggle/input/grammaratical-error-correction-dataset/train_updated.csv")

In [7]:
Grammaratical_Error.head()

,input,target
0,So I think we can not live if old people coul...,So I think we would not be alive if our ancest...
1,So I think we can not live if old people coul...,So I think we could not live if older people d...
2,So I think we can not live if old people coul...,So I think we can not live if old people could...
3,So I think we can not live if old people coul...,So I think we can not live if old people can n...
4,For not use car .,Not for use with a car .


In [8]:
Grammaratical_Error.tail()

,input,target
3011,Sceene of violence can affect on them .,Scenes of violence can affect them .
3012,While the communities in general have reckone...,The communities in general have reckoned that ...
3013,While the communities in general have reckone...,While the communities in general have reckoned...
3014,While the communities in general have reckone...,While the communities in general have recogniz...
3015,While the communities in general have reckone...,While the communities in general believe that ...


In [9]:
Grammaratical_Error.shape

(3016, 2)

In [10]:
Grammaratical_Error.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3016 entries, 0 to 3015
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input   3016 non-null   object
 1   target  3016 non-null   object
dtypes: object(2)
memory usage: 47.2+ KB


In [11]:
Grammaratical_Error.describe().T

,count,unique,top,freq
input,3016,754,While the communities in general have reckone...,4
target,3016,2500,I was upset and depressed .,4


`strat the model`




using model (vennify/t5-base-grammar-correction)

In [12]:
inputs = Grammaratical_Error['input'].tolist()
targets = Grammaratical_Error['target'].tolist()

In [20]:
# Load model and tokenizer
model_name = 'vennify/t5-base-grammar-correction'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [14]:
def encode_data(inputs, targets, tokenizer, max_length=512):
    input_encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    target_encodings = tokenizer(targets, truncation=True, padding=True, max_length=max_length, return_tensors="pt")

    return input_encodings, target_encodings

input_encodings, target_encodings = encode_data(inputs, targets, tokenizer)


##`إنشاء مجموعة البيانات:`

In [15]:
class GrammarCorrectionDataset(Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.input_encodings['input_ids'])

    def __getitem__(self, idx):
        input_data = {key: val[idx].clone().detach() for key, val in self.input_encodings.items()}
        target_data = self.target_encodings['input_ids'][idx].clone().detach()
        return input_data, target_data

dataset = GrammarCorrectionDataset(input_encodings, target_encodings)


dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [16]:
def calculate_accuracy(predictions, labels):
    pred_ids = torch.argmax(predictions, dim=-1)
    correct = (pred_ids == labels).float()
    return correct.sum() / len(correct)

In [27]:
optimizer = AdamW(model.parameters(), lr=1e-5)

GradScaler: هو كائن يقوم بتوسيع التدرجات لتجنب فقدان الدقة عند استخدام float16.
autocast(): يقوم بتحويل العمليات الحسابية ذات الكثافة العالية إلى دقة 16 بت تلقائيًا.
scaler.scale() و scaler.step(): يتم استخدام هذه العمليات لتوسيع التدرجات وتحديث المعلمات بناءً على تلك التدرجات.

In [28]:
# 7. إعداد Mixed Precision
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()

# 8. التدريب مع Mixed Precision
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_accuracy = 0

    for batch in dataloader:
        optimizer.zero_grad()
        input_data, labels = batch
        input_data = {key: val.to(device) for key, val in input_data.items()}
        labels = labels.to(device)

        # تفعيل Mixed Precision
        with autocast():
            outputs = model(input_ids=input_data['input_ids'], attention_mask=input_data['attention_mask'], labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        # حساب الدقة
        accuracy = calculate_accuracy(logits, labels)

        # استخدام GradScaler لتوسيع التدرجات
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    avg_train_loss = total_loss / len(dataloader)
    avg_train_accuracy = total_accuracy / len(dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss}, Accuracy: {avg_train_accuracy}")


/tmp/ipykernel_36/1687180659.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_36/1687180659.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/3, Loss: 0.08443651350328714, Accuracy: 95.83123342175067
Epoch 2/3, Loss: 0.08111437121085052, Accuracy: 95.90019893899205
Epoch 3/3, Loss: 0.0783639473578737, Accuracy: 95.94960212201592


In [34]:
def correct_text(text, model, tokenizer, max_length=512):
    # التأكد من أن النموذج على نفس الجهاز الذي سيتم عليه التنبؤ
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # ترميز النص المُدخل
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=True).to(device)
    
    # توليد النص المصحح باستخدام النموذج
    with torch.no_grad():
        output = model.generate(
            input_ids, 
            max_length=max_length, 
            num_beams=4, 
            early_stopping=True,
            decoder_start_token_id=tokenizer.pad_token_id
        )
    
    # فك ترميز النص المصحح
    corrected_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return corrected_text

# اختبار النموذج
test_sentence = "She have finished her homeيؤيؤيؤworkيبيب"
corrected_sentence = correct_text(test_sentence, model, tokenizer)
print("Corrected Sentence:", corrected_sentence)


Corrected Sentence: She has finished her homework.


In [35]:
model.save_pretrained("path_to_save_model")
tokenizer.save_pretrained("path_to_save_tokenizer")

('path_to_save_tokenizer/tokenizer_config.json',
 'path_to_save_tokenizer/special_tokens_map.json',
 'path_to_save_tokenizer/spiece.model',
 'path_to_save_tokenizer/added_tokens.json')